In [1]:
import pandas as panda

import numpy as np

import datetime, time

from matplotlib.pyplot import plot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns

import dask.dataframe as dd
from sklearn.linear_model import LinearRegression, RANSACRegressor, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

In [2]:
historical_transaction_data_path = 'all/historical_transactions.csv'
new_transaction_data_path = 'all/new_merchant_transactions.csv'

In [3]:
historical_transaction_data = panda.read_csv(historical_transaction_data_path)

In [4]:
new_transaction_data = panda.read_csv(new_transaction_data_path)

In [5]:
# new_transaction_data.head()

In [6]:
all_transaction = panda.concat([new_transaction_data, historical_transaction_data])

In [7]:
# len(all_transaction)

In [8]:
# all_transaction.isnull().sum()

In [9]:
## category_3, _2 are categorical variables. replacing missing values with max appearing values for each

## not doing anything with merchant_id yet. since its alphabetical we will remove it. and for our round 2 we will not be considering
## any data from merchant .csv


all_transaction['category_2'].fillna(all_transaction.category_2.value_counts().idxmax(), inplace= True)
all_transaction['category_3'].fillna(all_transaction.category_3.value_counts().idxmax(), inplace= True)


In [10]:
# all_transaction_1.head()
# all_transaction['category_2'].isnull().sum()

In [11]:
# all_transaction['category_3'].isnull().sum()

In [12]:
## lets get the total number of transactions for each card

total_transaction = all_transaction.groupby(['card_id'])['city_id'].count()

total_transaction = total_transaction.reset_index()

total_transaction.columns = ['card_id', 'total_no_of_transaction']

total_transaction_merged = all_transaction.merge(total_transaction, how='inner', on='card_id')

print(len(total_transaction_merged), ', earlier length: ', len(all_transaction))



31075392 , earlier length:  31075392


In [13]:
## we will now add columsn for how many times a card has been rejected and how many times it has been approved

# total_transaction_merged.head()



In [14]:
temp= total_transaction_merged.authorized_flag.str.get_dummies()



In [15]:
temp.head()

,N,Y
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [16]:
total_transaction_merged[['N','Y']] = temp

In [17]:
tt = total_transaction_merged.groupby(['card_id'])['N'].sum()

tt = tt.reset_index()



In [18]:
tt['card_id'] = tt.card_id.apply(lambda x: x.upper().strip())
tt

,card_id,N
0,C_ID_00007093C1,35
1,C_ID_0001238066,3
2,C_ID_0001506EF0,4
3,C_ID_0001793786,27
4,C_ID_000183FDDA,7
5,C_ID_00024E244B,17
6,C_ID_0002709B5A,6
7,C_ID_00027503E2,18
8,C_ID_000298032A,2
9,C_ID_0002BA3C2E,15


In [19]:
total_transaction_merged['card_id'] = total_transaction_merged.card_id.apply(lambda x: x.upper().strip())

# total_transaction_merged

In [20]:
no_count_merged = total_transaction_merged.merge(tt, how='left', on='card_id')

In [21]:

# total_transaction_merged = total_transaction_merged[[i for i in total_transaction_merged.columns.tolist() if i not in ['N','Y']]]
# no_count_merged_1 = total_transaction_merged[[i for i in total_transaction_merged.columns.tolist() if i not in ['N','Y']]].merge(tt, how='left', on='card_id')

In [22]:
tt.head()

,card_id,N
0,C_ID_00007093C1,35
1,C_ID_0001238066,3
2,C_ID_0001506EF0,4
3,C_ID_0001793786,27
4,C_ID_000183FDDA,7


In [23]:
len(total_transaction_merged)

31075392

In [24]:
no_count_merged['total_rejects'] = no_count_merged['N_y'] 

In [25]:
no_count_merged['total_accepted'] = no_count_merged['total_no_of_transaction'] -no_count_merged['total_rejects']

In [26]:
# no_count_merged.head()

##### at this point we have total no of transactions, total no of rejected and total no of approved transactions

##### we would want another column that says total purchase amount

In [27]:
# historical_store = panda.HDFStore('all/yes_no_total_added.h5')
# historical_store['df'] = no_count_merged
no_count_merged.to_csv('all/yes_no_merged.csv')
# no_count_merged[0:25000].to_pickle('all/yes_no_merged_25000.pkl')

In [28]:
# no_count_merged[25000:300000].to_pickle('all/yes_no_merged_300000.pkl')

In [29]:
# no_count_merged[300000:1000000].to_pickle('all/yes_no_merged_1mil.pkl')

In [30]:
# no_count_merged[1000000:2000000].to_pickle('all/yes_no_merged_2mil.pkl')

In [31]:
# no_count_merged[2000000:3000000].to_pickle('all/yes_no_merged_3mil.pkl')

In [32]:
# no_count_merged[2000000:2000000].to_pickle('all/yes_no_merged_4mil.pkl')